# Working with functions in Azure OpenAI
This notebook shows how to use the Chat Completions API in combination with functions to extend the current capabilities of GPT models. GPT models, do not inherently support real-time interaction with external systems, databases, or files. However, functions can be used to do so.

Overview: <br>
`functions` is an optional parameter in the Chat Completion API which can be used to provide function specifications. This allows models to generate function arguments for the specifications provided by the user. 

Note: The API will not execute any function calls. Executing function calls using the outputed argments must be done by developers. 

## Setup

In [7]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version ="2023-07-01-preview"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")

## 1.0 Test functions

This code calls the model with the user query and the set of functions defined in the functions parameter. The model then can choose if it calls a function. If a function is called, the content will be in a strigified JSON object. The function call that should be made and arguments are location in:  response[`choices`][0][`function_call`].

In [23]:
deployment_id="gpt-4-32k"

In [24]:
def get_function_call(messages, function_call = "auto"):
    # Define the functions to use
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    ]

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = openai.ChatCompletion.create(
        engine = deployment_id,
        messages=messages,
        functions=functions,
        function_call=function_call, 
    )

    return response

### Forcing the use of a specific function or no function
By changing the value of the `functions` parameter you can allow the model to decide what function to use, force the model to use a specific function, or force the model to use no function.

In [25]:
first_message = [{"role": "user", "content": "What's the weather like in San Francisco?"}]
# 'auto' : Let the model decide what function to call
print("Let the model decide what function to call:")
print (get_function_call(first_message, "auto"))

# 'none' : Don't call any function 
print("Don't call any function:")
print (get_function_call(first_message, "none"))

# force a specific function call
print("Force a specific function call:")
print (get_function_call(first_message, function_call={"name": "get_current_weather"}))

Let the model decide what function to call:
{
  "id": "chatcmpl-7oQgKcJmASLzUbq73GXpsFO1QJoFZ",
  "object": "chat.completion",
  "created": 1692253320,
  "model": "gpt-4-32k",
  "prompt_annotations": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "function_call",
      "message": {
        "role": "assistant",
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco\"\n}"
        }
      },
      "content_filter_results": {}
    }
  ],
  "usage": {
    "completion_to

## 2.0 Defining functions
Now that we know how to work with functions, let's define some functions in code so that we can walk through the process of using functions end to end.

### Function #1: Get current time

In [30]:
import pytz
from datetime import datetime

def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

In [27]:
get_current_time("America/New_York")

'02:23:35 AM'

## 3.0 Calling a function using GPT

Steps for Function Calling: 

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call a function; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may generate invalid JSON or hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

### 3.1 Describe the functions so that the model knows how to call them

In [40]:
functions = [
        {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                    }
                },
                "required": ["location"],
            },
        },
        
    ]

available_functions = {
            "get_current_time": get_current_time
        } 

### 3.2 Define a helper function to validate the function call
It's possible that the models could generate incorrect function calls so it's important to validate the calls. Here we define a simple helper function to validate the function call although you could apply more complex validation for your use case.

In [34]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [37]:
import json

In [41]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto", 
    )
    response_message = response["choices"][0]["message"]


    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)
        
        print("Output of function call:")
        print(function_response)
        print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response

In [42]:
messages = [{"role": "user", "content": "What time is it in New York?"}]
assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
print(assistant_response['choices'][0]['message'])

Recommended Function call:
{
  "name": "get_current_time",
  "arguments": "{\n  \"location\": \"America/New_York\"\n}"
}

Output of function call:
02:28:59 AM

Messages in second request:
{'role': 'user', 'content': 'What time is it in New York?'}
{'role': 'assistant', 'name': 'get_current_time', 'content': '{\n  "location": "America/New_York"\n}'}
{'role': 'function', 'name': 'get_current_time', 'content': '02:28:59 AM'}

{
  "role": "assistant",
  "content": "It is currently 2:28 AM in New York."
}
